In [1]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Load dataset
df = pd.read_csv(r"D:\Stroke_prediction_project\preprocessed_stroke_data.csv")

# --- Encode categorical variables ---
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Features & Target
X = df.drop("Stroke", axis=1)
y = df["Stroke"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train logistic regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)


# Save model + scaler + feature columns
with open(r"D:\Stroke_prediction_project\models\logistic_regression.pkl", "wb") as f:
    pickle.dump((model, scaler, X.columns.tolist()), f)

